In [1]:
from transformers import Speech2TextForConditionalGeneration, Speech2TextProcessor, Speech2TextModel, Speech2TextTokenizer
import torch
import torchaudio
import numpy as np

# from auditory_cortex.regression import transformer_regression
from auditory_cortex.dataset import Neural_Data
from auditory_ctx.models.speech_recognition import SpeechRecognitionModel


In [2]:
# data_path = '/Users/akshita/Documents/Research/Makin/data'
# subject = '200206'
# reg = transformer_regression(data_path, subject)

# def get_beta_eta(layer=0):
#     train_list = np.arange(1,451).tolist()
#     z_vals_train, n_vals_train = reg.get_layer_values_and_spikes(layer=layer, win=80, sent_list=train_list)
#     # print(z_vals_train.shape, n_vals_train.shape)
    
#     beta = np.linalg.solve(z_vals_train.T.dot(z_vals_train), (z_vals_train.T).dot(n_vals_train))
#     return beta, n_vals_train

In [3]:
dir = '/Users/akshita/Documents/Research/Makin/data'
subject = '200206'
neural_data = Neural_Data(dir, subject)

In [4]:
# model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
# processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

# model = Speech2TextModel.from_pretrained("facebook/s2t-small-librispeech-asr")
# processor = Speech2TextTokenizer

cnn_layers= 3
rnn_layers= 5
rnn_dim= 512
n_class= 29
n_features= 128
stride= 2
dropout= 0.1
batch_size = 64
char_path = '/Users/akshita/Documents/Research/Makin/auditory-ctx/conf/char.space.27'

model = SpeechRecognitionModel(cnn_layers, rnn_layers, rnn_dim, n_class, n_features, stride, dropout)


checkpoint = torch.load('/Users/akshita/Documents/Research/Makin/outputs/Uni_GRU_model/epoch_600.pt', map_location=torch.device('cpu')) 

# checkpoint = torch.load("/scratch/gilbreth/akamsali/Research/Makin/outputs/long/models/epoch_245.pt")
model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])
aud = neural_data.audio(0)

In [5]:
spect = torchaudio.transforms.MelSpectrogram()
aud_tensor = torchaudio.load('/Users/akshita/Documents/Research/Makin/Auditory_Cortex/notebooks/test.wav')

/Users/akshita/opt/anaconda3/envs/research/lib/python3.9/site-packages/torchaudio/functional/functional.py:594: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [6]:
spectogram = spect(aud_tensor[0]).unsqueeze(1).transpose(0, 1)

In [7]:
class Hook():
    def __init__(self, model, layer_name, backward=False):

        layer = dict([*model.named_modules()])[layer_name]
        layer.__name__ = layer_name
        # self.hook_inp = spect_inp.register_backward_hook()
        # if backward == False:

        self.hook_fwd = layer.register_forward_hook(self.hook_fn_f)
        self.hook_bwd = layer.register_backward_hook(self.hook_fn_b)
        # else:

    def hook_fn_f(self, layer, input, output):
        # print("i", input)
        # print("0", output)

        self.input_f = input
        self.output_f = output#.requires_grad_(True)

    def hook_fn_b(self, layer, input, output):
        # print("i", input)
        # print("0", output)

        self.input_b = input
        self.output_b = output
    
    
    def close(self):
        self.hook.remove()

def get_gradients(net_in, net, layer):     
    # aud_processed = processor(net_in, padding=True, sampling_rate=16000, return_tensors="pt")['input_features']
    # print(aud_processed)
    net_in.requires_grad = True
    # op = torch.optim.SGD([aud_processed], lr=0.1, momentum=0.9)
    net.zero_grad()
    hook = Hook(net, layer, backward=True)
    net_out = net(net_in)
    # print(hook.output_f)
    loss = hook.output_f[0].mean()
    loss.backward()

    # print(hook.input_b, hook.output_b)
    # op.step()
    # print(net_in.grad)
    # print(net_in.grad, net_in.grad_fn)

    return net_in.grad.data

def dream(audio, net, layer, iterations, lr):

    for i in range(iterations):
        gradients = get_gradients(audio, net, layer)
        audio.data = audio.data + lr * gradients.data

    audio_out = audio.detach().cpu()
    
    return audio_out

In [8]:
model.eval()
dream(spectogram, model, "cnn", iterations=1, lr=1)

[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
/Users/akshita/opt/anaconda3/envs/research/lib/python3.9/site-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


tensor([[[[8.1675e-06, 2.1972e-05, 8.1675e-06,  ..., 2.1972e-05,
           8.1675e-06, 6.6713e-06],
          [5.4910e-05, 7.2356e-05, 6.0224e-05,  ..., 7.2559e-05,
           1.3822e-04, 1.0164e-04],
          [5.9077e-05, 8.5118e-05, 8.7691e-05,  ..., 8.6212e-05,
           5.0765e-04, 4.5477e-04],
          ...,
          [5.0684e-05, 7.0149e-05, 8.0419e-05,  ..., 7.4073e-05,
           5.9124e-05, 2.3174e-05],
          [3.9668e-05, 3.5637e-05, 1.7201e-05,  ..., 3.5897e-05,
           4.4854e-05, 3.0475e-05],
          [1.0664e-04, 5.3002e-05, 3.7897e-05,  ..., 4.6879e-05,
           4.0434e-05, 4.8521e-05]]]])

In [9]:
model_tf = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
# processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")


In [16]:
model_tf

Speech2TextForConditionalGeneration(
  (model): Speech2TextModel(
    (encoder): Speech2TextEncoder(
      (conv): Conv1dSubsampler(
        (conv_layers): ModuleList(
          (0): Conv1d(80, 1024, kernel_size=(5,), stride=(2,), padding=(2,))
          (1): Conv1d(512, 512, kernel_size=(5,), stride=(2,), padding=(2,))
        )
      )
      (embed_positions): Speech2TextSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0): Speech2TextEncoderLayer(
          (self_attn): Speech2TextAttention(
            (k_proj): Linear(in_features=256, out_features=256, bias=True)
            (v_proj): Linear(in_features=256, out_features=256, bias=True)
            (q_proj): Linear(in_features=256, out_features=256, bias=True)
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=256, out_features=2048, bias=True)
        

In [ ]:
dream(aud, model, "encoder.conv.conv_layers.1", iterations=1, lr=1)

In [ ]:
aud_processed.keys()

In [ ]:
from auditory_cortex.dataset import Neural_Data

# dir = '/depot/jgmakin/data/auditory_cortex/josh_data/data'
dir = '/Users/akshita/Documents/Research/Makin/data'
subject = '200206'
neural_data = Neural_Data(dir, subject)

In [ ]:
import torchaudio
import torch
import os
wav_path = '/Users/akshita/Documents/Research/Makin/data/wav_files'
wav_files = list(map(lambda x: wav_path +'/sentence_' + str(x+1) +'.wav', range(500)))

In [ ]:
class GetFeatures:
    def __init__(self, dir, subject, delay=0, model='transformer', VERBOSE=False):
        self.model_name = model
        if self.model_name == 'transformer':
            self.model = TransformerModel()
            self.neural_data = Neural_Data(dir, subject)

            self.get_transformer_features()
            if VERBOSE:
                print("Got transformer features")

            self.get_spikes()
            if VERBOSE:
                print("Got spikes")

    def translate(self, aud, fs = 16000):
        self.inputs_features = self.model.processor(aud,padding=True, sampling_rate=fs, return_tensors="pt").input_features
        generated_ids = self.model.model_extractor(self.inputs_features)

    def get_transformer_features(self):
        features = [{} for _ in range(len(self.model.layers))]
        for i in range(0,500):
            self.translate(self.neural_data.audio(i))

            for j, l in enumerate(self.model.layers):
                features[j][i] = self.model.model_extractor.features[l]

        self.all_hidden_features = features
        

    def get_spikes(self, delay=0):
        spikes = [{}, {}]
        for i in range(1,500):
            spikes[0][i] = self.neural_data.retrieve_spike_counts(sent=i, win=20, delay=delay, 
                                                                early_spikes=False,
                                                                model=self.model_name, 
                                                                offset=-0.25)
            spikes[1][i] = self.neural_data.retrieve_spike_counts(sent=i, win=40, delay=delay, 
                                                                early_spikes=False,
                                                                model=self.model_name,
                                                                offset=0.39)

        self.all_spikes_dict = spikes